In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_train = '/drive/dataset/'

In [ ]:
!pip install jsonlines pandas matplotlib nltk

In [ ]:
import jsonlines
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%cd   /content/drive/MyDrive/prueba_tenica_BC/

/content/drive/MyDrive/prueba_tenica_BC


In [ ]:
def load_data(dataset):
    with jsonlines.open(dataset, 'r') as reader:
        data = list(reader)
        list_messages = []
        list_labels = []
        for line in data:
            list_messages.extend(line['messages'])
            list_labels.extend(line['sender_labels'])
    dict_out = {'messages': list_messages, 'label': list_labels}
    df = pd.DataFrame(dict_out)
    return df

### Carga los dataset

In [ ]:
path_test = r'./dataset/test_diplomacy.jsonl'
path_train = r'./dataset/train_diplomacy.jsonl'
path_validation = r'./dataset/validation_diplomacy.jsonl'

In [ ]:
test=load_data(path_test)
train = load_data(path_train)
validation = load_data(path_validation)

### Valida los mensajes vacíos

In [ ]:
na_test = test[test.isna().any(axis=1)]
na_train = train[train.isna().any(axis=1)]
na_vali = validation[validation.isna().any(axis=1)]

print("cantidad de datos vacios de entrenamiento" ,na_train.shape[0])
print("cantidad de datos vacios de validacion" ,na_vali.shape[0])
print("cantidad de datos vacios de test" ,na_test.shape[0])

cantidad de datos vacios de entrenamiento 0
cantidad de datos vacios de validacion 0
cantidad de datos vacios de test 0


### filtrar mensajes con caracteres menores a 10

In [ ]:
train_raw = train[train['messages'].str.len() >= 10]
val_raw = validation[validation['messages'].str.len() >= 10]
test_raw = test[test['messages'].str.len() >= 10]

In [ ]:
train_raw.head()

,messages,label
0,Germany!\n\nJust the person I want to speak wi...,True
1,"You've whet my appetite, Italy. What's the sug...",True
3,It seems like there are a lot of ways that cou...,True
4,"Yeah, I can’t say I’ve tried it and it works, ...",True
5,I am just sensing that you don’t like this ide...,True


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

In [ ]:
train_raw['label_bin'] = train_raw['label'].apply(lambda x: 1 if x == False else 0)
test_raw['label_bin'] = test_raw['label'].apply(lambda x: 1 if x == False else 0)
val_raw['label_bin'] = val_raw['label'].apply(lambda x: 1 if x == False else 0)


train_raw['label_bin'] = train_raw['label_bin'].astype(float)
test_raw['label_bin'] = test_raw['label'].astype(float)
val_raw['label_bin'] = val_raw['label'].astype(float)

<ipython-input-16-7ae6aa7b96a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_raw['label_bin'] = train_raw['label'].apply(lambda x: 1 if x == False else 0)
<ipython-input-16-7ae6aa7b96a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_raw['label_bin'] = test_raw['label'].apply(lambda x: 1 if x == False else 0)
<ipython-input-16-7ae6aa7b96a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [ ]:
train_raw[train_raw['label']==False]

,messages,label,label_bin
132,I really don’t think that’s a fair description...,False,1.0
135,"We’re friends, right? I believe that every sin...",False,1.0
139,I have to say that I’m surprised that you feel...,False,1.0
144,Here’s the deal: I like you better than England.,False,1.0
158,But I don’t think there is anything wrong with...,False,1.0
...,...,...,...
13067,Not sure what the Italian is scheming. They to...,False,1.0
13087,Greetings Sultan!\n\nAs your neighbor I would ...,False,1.0
13089,Sounds good lets call a dmz in the black sea,False,1.0
13105,Just two of you. But their alliance will go fo...,False,1.0


In [ ]:
new_train = train_raw[['messages','label_bin']]
new_test = test_raw[['messages','label_bin']]
new_val = val_raw[['messages','label_bin']]

### Algunas variables definidas para luego ser usadas en el entrenamiento

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class diplomacy_data(Dataset):   #### SentimetDataset
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.messages ###Phrase
        self.targets = self.data.label_bin  ### sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
new_train = new_train.reset_index(drop=True)
new_test = new_test.reset_index(drop=True)
new_val = new_val.reset_index(drop=True)

In [ ]:
training_set = diplomacy_data(new_train, tokenizer, MAX_LEN)
testing_set = diplomacy_data(new_test, tokenizer, MAX_LEN)
vali_set = diplomacy_data(new_val, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
#vali_loader = DataLoader(vali_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.sigmoid(output)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

### Fine Tunning del modelo



In [ ]:
!pip install torcheval

  Using cached torcheval-0.0.7-py3-none-any.whl (179 kB)


In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from torcheval.metrics.functional import binary_f1_score
from torchmetrics.classification import BinaryF1Score

In [ ]:
weights = torch.tensor([10.0])
loss_function = torch.nn.BCELoss(weight=weights)
loss_function.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    tr_f1_score = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        #targets = targets.unsqueeze(1)

        outputs = model(ids, mask, token_type_ids)
        print(outputs)
        print(targets)
        loss = loss_function(outputs, targets.float().unsqueeze(1))
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)
        #from sklearn.metrics import f1_score
        # f1_score_mic = BinaryF1Score(big_idx,targets,threshold=0.5)
        # tr_f1_score += f1_score_mic.item()

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            # f1_score = tr_f1_score/nb_tr_steps
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")
            # print(f"Training F1_Score per 5000 steps: {f1_score}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    epoch_f1_score = tr_f1_score/nb_tr_steps
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")
    print(f"Training F1 Score Epoch: {epoch_f1_score}")

    return

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

tensor([[0.0430],
        [0.0518],
        [0.0253],
        [0.0578],
        [0.0417],
        [0.1230],
        [0.0859],
        [0.1334]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


TypeError: BinaryF1Score.__init__() got multiple values for argument 'threshold'